In [1]:
from pydruid.client import *
from pydruid.utils.aggregators import *
from pydruid.utils.postaggregator import *
from pydruid.utils.filters import *
from pydruid.utils.having import Having



# Groupby query getting # Records per Day

In [2]:
query = PyDruid('http://bislave02.dev3.bloombergpolarlake.com:8082', 'druid/v2/')
 
 
weekly_counts = query.groupby(
    datasource = "bpl_bdp_publication_dev3.bpl_market_master_druid",
    granularity = "day",
    intervals = "2018-05-01T00:00/2019-10-28T23:59",
    aggregations = {"num_rows": count(None)}
)
 
print (weekly_counts)  # Do this if you want to see the raw JSON

[{'version': 'v1', 'timestamp': '2019-08-25T00:00:00.000Z', 'event': {'num_rows': 7517}}, {'version': 'v1', 'timestamp': '2019-08-31T00:00:00.000Z', 'event': {'num_rows': 748}}, {'version': 'v1', 'timestamp': '2019-09-01T00:00:00.000Z', 'event': {'num_rows': 742}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'num_rows': 739}}, {'version': 'v1', 'timestamp': '2019-09-03T00:00:00.000Z', 'event': {'num_rows': 501}}, {'version': 'v1', 'timestamp': '2019-09-06T00:00:00.000Z', 'event': {'num_rows': 501}}, {'version': 'v1', 'timestamp': '2019-09-07T00:00:00.000Z', 'event': {'num_rows': 335}}, {'version': 'v1', 'timestamp': '2019-09-08T00:00:00.000Z', 'event': {'num_rows': 310}}, {'version': 'v1', 'timestamp': '2019-09-09T00:00:00.000Z', 'event': {'num_rows': 310}}, {'version': 'v1', 'timestamp': '2019-09-10T00:00:00.000Z', 'event': {'num_rows': 310}}, {'version': 'v1', 'timestamp': '2019-09-12T00:00:00.000Z', 'event': {'num_rows': 308}}]


# Export query result to Pandas DataFrame

In [3]:

df = query.export_pandas() # Client will import Pandas, no need to do so separately.
 
df = df.drop('timestamp', axis=1)  # Don't need the timestamp column here
 
df.index = range(1, len(df)+1)  # Get a naturally numbered index
 
print (df)
 
#df.plot(x='num_rows', kind='bar')
 
#plt.show()

    num_rows
1       7517
2        748
3        742
4        739
5        501
6        501
7        335
8        310
9        310
10       310
11       308


# Total number of records accross all time

In [4]:
 
query = PyDruid('http://bislave02.dev3.bloombergpolarlake.com:8082', 'druid/v2/')
 
 
weekly_counts = query.groupby(
    datasource = "bpl_bdp_publication_dev3.bpl_market_master_druid",
    granularity = "all",
    intervals = "2018-05-01T00:00/2020-08-28T23:59",
    aggregations = {"num_rows": count(None)}
)
 
print (weekly_counts)  # Do this if you want to see the raw JSON

[{'version': 'v1', 'timestamp': '2018-05-01T00:00:00.000Z', 'event': {'num_rows': 12321}}]


# TopN Query to count the # records by bpl_classifier_i 
Note that TopN queries are an approximation only.

In [5]:

 
query = PyDruid('http://bislave02.dev3.bloombergpolarlake.com:8082', 'druid/v2/')
 
 
top_langs = query.topn(
    datasource = "bpl_bdp_publication_dev3.bpl_market_master_druid",
    granularity = "all",
    intervals = "2019-05-01T00:00/2019-10-28T23:59",
    dimension = "bpl_classifier_i",
    aggregations = {"updates": count("*")},
    #filter=Dimension("id_bb_global") == "BBG00JPQ7J31",
    metric = "updates",
    threshold = 10000
)
 
print (top_langs)  # Do this if you want to see the raw JSON

[{'timestamp': '2019-08-25T00:00:00.000Z', 'result': [{'updates': 7053, 'bpl_classifier_i': 'E'}, {'updates': 3236, 'bpl_classifier_i': 'F'}, {'updates': 506, 'bpl_classifier_i': 'T'}, {'updates': 412, 'bpl_classifier_i': 'X'}, {'updates': 323, 'bpl_classifier_i': 'P'}, {'updates': 283, 'bpl_classifier_i': 'I'}, {'updates': 232, 'bpl_classifier_i': 'O'}, {'updates': 189, 'bpl_classifier_i': 'N'}, {'updates': 87, 'bpl_classifier_i': 'L'}]}]


# Max ? broken down by bpl_classifier_i and bpl_classifier_ii by month
## Nothing available to aggregate

In [7]:
 
#query = PyDruid('http://bislave02.dev3.bloombergpolarlake.com:8082', 'druid/v2/')
 
 
#weekly_counts = query.groupby(
#    datasource = "ssb_druid.bpl_pricing_druid_cube",
#    granularity = "month",
#    intervals = "2018-05-01T00:00/2018-08-28T23:59",
#    aggregations = {"max_price": doublemax("px_last")},
#    dimensions=["bpl_classifier_i", "bpl_classifier_ii"]
#)
 
#print weekly_counts  # Do this if you want to see the raw JSON

# Post Aggregator Example 
Not a realistic example - just divides monthly count by 31

In [8]:
 
query = PyDruid('http://bislave02.dev3.bloombergpolarlake.com:8082', 'druid/v2/')
 
 
weekly_counts = query.groupby(
    datasource = "bpl_bdp_publication_dev3.bpl_market_master_druid",
    granularity = "month",
    intervals = "2018-05-01T00:00/2020-08-28T23:59",
    aggregations = {"num_rows": count(None)},
    post_aggregations={"average_daily":(Field("num_rows")/Const(31))}
)
 
print (weekly_counts)  # Do this if you want to see the raw JSON

[{'version': 'v1', 'timestamp': '2019-08-01T00:00:00.000Z', 'event': {'num_rows': 8265, 'average_daily': 266.61290322580646}}, {'version': 'v1', 'timestamp': '2019-09-01T00:00:00.000Z', 'event': {'num_rows': 4056, 'average_daily': 130.83870967741936}}]


# Check are there any markets updated more than 4 times same week

In [9]:
 
query = PyDruid('http://bislave02.dev3.bloombergpolarlake.com:8082', 'druid/v2/')
 
 
weekly_counts = query.groupby(
    datasource = "bpl_bdp_publication_dev3.bpl_market_master_druid",
    granularity = "week",
    intervals = "2018-05-01T00:00/2020-08-28T23:59",
    aggregations = {"num_rows": count(None)},
    dimensions = ["id_bpl_market"],
    having = Having(type="greaterThan", aggregation="num_rows", value=4),
        limit_spec={
        "type": "default",
        "limit": 25,
        "columns" : ["num_rows"]
    }
)
 
print (weekly_counts)  # Do this if you want to see the raw JSON

[{'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_market': '10000002', 'num_rows': 5}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_market': '10000036', 'num_rows': 5}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_market': '10000070', 'num_rows': 5}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_market': '10000344', 'num_rows': 5}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_market': '10000336', 'num_rows': 5}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_market': '10000328', 'num_rows': 5}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_market': '10000090', 'num_rows': 5}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_market': '10000084', 'num_rows': 5}}, {'version': 'v1', 'timestamp': '2019-09-02T00:00:00.000Z', 'event': {'id_bpl_ma

# Any records updated more than once a day in

In [10]:
query = PyDruid('http://bislave02.dev3.bloombergpolarlake.com:8082', 'druid/v2/')
 
 
weekly_counts = query.groupby(
    datasource = "bpl_bdp_publication_dev3.bpl_market_master_druid",
    granularity = "day",
    intervals = "2018-05-01T00:00/2020-08-28T23:59",
    aggregations = {"num_rows": count(None)},
    dimensions = ["id_bpl_market"],
    having = Having(type="greaterThan", aggregation="num_rows", value=1),
        limit_spec={
        "type": "default",
        "limit": 50,
        "columns" : ["num_rows"]
    }
)
 
print (weekly_counts)  # Do this if you want to see the raw JSON

[]


# Attempt to query unique file IDs per day
Not Working currently !
A topN query might be better.

In [11]:
 
query = PyDruid('http://biambari01.dev3.bloombergpolarlake.com:8082', 'druid/v2/')
 
 
weekly_counts = query.timeseries(
    datasource = "ssb_druid.bloomberg_mtge_druid_cube",
    granularity = "week",
    intervals = "2018-01-01T00:00/2018-02-28T23:59",
    aggregations =  {"unique_files":countDistinct("system_id")},
)
 
print weekly_counts  # Do this if you want to see the raw JSON

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(weekly_counts  # Do this if you want to see the raw JSON)? (<ipython-input-11-35d5166c7fc1>, line 12)